In [1]:
import pandas as pd
import numpy as np

## 여자 거주인구 전처리

In [2]:
data1 = pd.read_csv("C:/Users/user/Desktop/03_거주인구_RESIDENT_POP_female.csv", encoding='cp949')

In [3]:
data1.head()

,base_year,base_month,city,emd_cd,emd_nm,sex,resid_reg_pop,foreign_pop,total_pop
0,2018,1,제주시,50110590,건입동,여성,4715,82,4797
1,2018,1,제주시,50110256,구좌읍,여성,7609,184,7793
2,2018,1,서귀포시,50130253,남원읍,여성,9306,200,9506
3,2018,1,제주시,50110660,노형동,여성,27083,768,27851
4,2018,1,서귀포시,50130590,대륜동,여성,6637,95,6732


In [4]:
# 필요없는 열 삭제 (날짜, 행정동코드, 행정동명, 배출량만 남김)
data1_pre = data1.drop(['city', 'emd_nm', 'sex', 'total_pop'], 1)

# 열 이름 바꾸기 (여_거주, 여_외국, 남_거주, 남_외국)
data1_pre.rename(columns={'resid_reg_pop':'fe_resid_reg_pop'}, inplace=True)
data1_pre.rename(columns={'foreign_pop':'fe_foreign_pop'}, inplace=True)
data1_pre

,base_year,base_month,emd_cd,fe_resid_reg_pop,fe_foreign_pop
0,2018,1,50110590,4715,82
1,2018,1,50110256,7609,184
2,2018,1,50130253,9306,200
3,2018,1,50110660,27083,768
4,2018,1,50130590,6637,95
...,...,...,...,...,...
1801,2021,6,50130320,6033,123
1802,2021,6,50110310,4531,100
1803,2021,6,50110250,10341,1140
1804,2021,6,50110600,12238,161


In [5]:
# 년, 월 병합
#df.insert(1,"base_date",['Lahore','Dehli','New York'],True)

data1_pre['base_year']= data1_pre['base_year'].astype('str')
data1_pre['base_month']= data1_pre['base_month'].astype('str')
data1_pre['fe_resid_reg_pop']= data1_pre['fe_resid_reg_pop'].astype('int')
data1_pre['fe_foreign_pop']= data1_pre['fe_foreign_pop'].astype('int')

In [6]:
data1_pre['base_date'] = data1_pre['base_year'] + "-" + data1_pre['base_month']

data1_pre = data1_pre.drop(['base_year', 'base_month'], 1)
data1_pre2 = data1_pre[['base_date', 'emd_cd', 'fe_resid_reg_pop', 'fe_foreign_pop']]
data1_pre2

,base_date,emd_cd,fe_resid_reg_pop,fe_foreign_pop
0,2018-1,50110590,4715,82
1,2018-1,50110256,7609,184
2,2018-1,50130253,9306,200
3,2018-1,50110660,27083,768
4,2018-1,50130590,6637,95
...,...,...,...,...
1801,2021-6,50130320,6033,123
1802,2021-6,50110310,4531,100
1803,2021-6,50110250,10341,1140
1804,2021-6,50110600,12238,161


In [7]:
# 각 열을 하나의 type으로 통일

# 날짜데이터 datetime으로 변환
data1_pre2['base_date'] = pd.to_datetime(data1_pre2['base_date'], format="%Y-%m-%d")

data1_pre2.set_index(data1_pre2['base_date'], inplace=True)

# "알수없음" 삭제
#idx = data2_pre2[data2_pre2['emd_cd'] == "XXXXXXXX"].index
#data2_pre2 = data2_pre2.drop(idx)

# 행정동코드 int형으로 변환
data1_pre2['emd_cd'] = pd.to_numeric(data1_pre2['emd_cd'])

In [8]:
# 행정동코드 매개변수로 받아서 해당 구역의 배출량을 월별로 정리
def monthly_output(emdcd):
    emd_data = data1_pre2[data1_pre2['emd_cd'] == emdcd]    # 해당 행정동 데이터만 추출
    emd_data_monthly = emd_data.resample('M', on='base_date').mean()    # 월별로 합산
    emd_data_monthly['emd_cd'] = emdcd
    return emd_data_monthly

In [9]:
emd_cd_ary = [50110250, 50110253, 50110256, 50110259, 50110310, 50110320, 50110330, 50110510, 50110520, 50110530, 50110540, 50110550, 50110560, 50110570, 50110580, 50110590, 50110600, 50110610, 50110620, 50110630, 50110640, 50110650, 50110660, 50110670, 50110680, 50110690, 50130250, 50130253, 50130259, 50130310, 50130320, 50130510, 50130520, 50130530, 50130540, 50130550, 50130560, 50130570, 50130580, 50130590, 50130600, 50130610, 50130620]

data1_result = pd.DataFrame(index=range(0,0), columns=['base_date', 'emd_cd', 'fe_resid_reg_pop', 'fe_foreign_pop'])

for i in emd_cd_ary:
    tmp = monthly_output(i)
    data1_result = data1_result.append(tmp)

data1_result['base_date'] = data1_result.index  
data1_result

,base_date,emd_cd,fe_resid_reg_pop,fe_foreign_pop
2018-01-31,2018-01-31,50110250,10081,1338
2018-02-28,2018-02-28,50110250,10089,1337
2018-03-31,2018-03-31,50110250,10136,1328
2018-04-30,2018-04-30,50110250,10168,1357
2018-05-31,2018-05-31,50110250,10214,1368
...,...,...,...,...
2021-02-28,2021-02-28,50130620,1904,137
2021-03-31,2021-03-31,50130620,1918,137
2021-04-30,2021-04-30,50130620,1904,135
2021-05-31,2021-05-31,50130620,1910,135


## 남자 거주인구 전처리

In [10]:
data2 = pd.read_csv("C:/Users/user/Desktop/03_거주인구_RESIDENT_POP_male.csv", encoding='cp949')

In [11]:
data2.head()

,base_year,base_month,city,emd_cd,emd_nm,sex,resid_reg_pop,foreign_pop,total_pop
0,2018,1,제주시,50110590,건입동,남성,5085,146,5231
1,2018,1,제주시,50110256,구좌읍,남성,7965,368,8333
2,2018,1,서귀포시,50130253,남원읍,남성,9806,428,10234
3,2018,1,제주시,50110660,노형동,남성,26286,821,27107
4,2018,1,서귀포시,50130590,대륜동,남성,6836,83,6919


In [12]:
# 필요없는 열 삭제 (날짜, 행정동코드, 행정동명, 배출량만 남김)
data2_pre = data2.drop(['city', 'emd_nm', 'sex', 'total_pop'], 1)

# 열 이름 바꾸기 (여_거주, 여_외국, 남_거주, 남_외국)
data2_pre.rename(columns={'resid_reg_pop':'m_resid_reg_pop'}, inplace=True)
data2_pre.rename(columns={'foreign_pop':'m_foreign_pop'}, inplace=True)
data2_pre

,base_year,base_month,emd_cd,m_resid_reg_pop,m_foreign_pop
0,2018,1,50110590,5085,146
1,2018,1,50110256,7965,368
2,2018,1,50130253,9806,428
3,2018,1,50110660,26286,821
4,2018,1,50130590,6836,83
...,...,...,...,...,...
1801,2021,6,50130320,6314,211
1802,2021,6,50110310,4627,237
1803,2021,6,50110250,10891,2090
1804,2021,6,50110600,12062,169


In [13]:
# 년, 월 병합
#df.insert(1,"base_date",['Lahore','Dehli','New York'],True)

data2_pre['base_year']= data2_pre['base_year'].astype('str')
data2_pre['base_month']= data2_pre['base_month'].astype('str')
data2_pre['m_resid_reg_pop']= data2_pre['m_resid_reg_pop'].astype('int')
data2_pre['m_foreign_pop']= data2_pre['m_foreign_pop'].astype('int')

In [14]:
data2_pre['base_date'] = data2_pre['base_year'] + "-" + data2_pre['base_month']

data2_pre = data2_pre.drop(['base_year', 'base_month'], 1)
data2_pre2 = data2_pre[['base_date', 'emd_cd', 'm_resid_reg_pop', 'm_foreign_pop']]
data2_pre2

,base_date,emd_cd,m_resid_reg_pop,m_foreign_pop
0,2018-1,50110590,5085,146
1,2018-1,50110256,7965,368
2,2018-1,50130253,9806,428
3,2018-1,50110660,26286,821
4,2018-1,50130590,6836,83
...,...,...,...,...
1801,2021-6,50130320,6314,211
1802,2021-6,50110310,4627,237
1803,2021-6,50110250,10891,2090
1804,2021-6,50110600,12062,169


In [15]:
# 각 열을 하나의 type으로 통일

# 날짜데이터 datetime으로 변환
data2_pre2['base_date'] = pd.to_datetime(data2_pre2['base_date'], format="%Y-%m-%d")

data2_pre2.set_index(data2_pre2['base_date'], inplace=True)

# "알수없음" 삭제
#idx = data2_pre2[data2_pre2['emd_cd'] == "XXXXXXXX"].index
#data2_pre2 = data2_pre2.drop(idx)

# 행정동코드 int형으로 변환
data2_pre2['emd_cd'] = pd.to_numeric(data2_pre2['emd_cd'])

In [16]:
# 행정동코드 매개변수로 받아서 해당 구역의 배출량을 월별로 정리
def monthly_output2(emdcd):
    emd_data = data2_pre2[data2_pre2['emd_cd'] == emdcd]    # 해당 행정동 데이터만 추출
    emd_data_monthly = emd_data.resample('M', on='base_date').mean()    # 월별로 합산
    emd_data_monthly['emd_cd'] = emdcd
    return emd_data_monthly

In [17]:
emd_cd_ary = [50110250, 50110253, 50110256, 50110259, 50110310, 50110320, 50110330, 50110510, 50110520, 50110530, 50110540, 50110550, 50110560, 50110570, 50110580, 50110590, 50110600, 50110610, 50110620, 50110630, 50110640, 50110650, 50110660, 50110670, 50110680, 50110690, 50130250, 50130253, 50130259, 50130310, 50130320, 50130510, 50130520, 50130530, 50130540, 50130550, 50130560, 50130570, 50130580, 50130590, 50130600, 50130610, 50130620]

data2_result = pd.DataFrame(index=range(0,0), columns=['base_date', 'emd_cd', 'm_resid_reg_pop', 'm_foreign_pop'])

for i in emd_cd_ary:
    tmp2 = monthly_output2(i)
    data2_result = data2_result.append(tmp2)

data2_result['base_date'] = data2_result.index  
data2_result

,base_date,emd_cd,m_resid_reg_pop,m_foreign_pop
2018-01-31,2018-01-31,50110250,10808,2192
2018-02-28,2018-02-28,50110250,10796,2205
2018-03-31,2018-03-31,50110250,10860,2208
2018-04-30,2018-04-30,50110250,10868,2213
2018-05-31,2018-05-31,50110250,10898,2235
...,...,...,...,...
2021-02-28,2021-02-28,50130620,2013,119
2021-03-31,2021-03-31,50130620,2007,116
2021-04-30,2021-04-30,50130620,2004,118
2021-05-31,2021-05-31,50130620,2021,112


## 남, 여 데이터 합치기

In [18]:
merge_inner = pd.merge(data1_result, data2_result)
merge_inner

,base_date,emd_cd,fe_resid_reg_pop,fe_foreign_pop,m_resid_reg_pop,m_foreign_pop
0,2018-01-31,50110250,10081,1338,10808,2192
1,2018-02-28,50110250,10089,1337,10796,2205
2,2018-03-31,50110250,10136,1328,10860,2208
3,2018-04-30,50110250,10168,1357,10868,2213
4,2018-05-31,50110250,10214,1368,10898,2235
...,...,...,...,...,...,...
1801,2021-02-28,50130620,1904,137,2013,119
1802,2021-03-31,50130620,1918,137,2007,116
1803,2021-04-30,50130620,1904,135,2004,118
1804,2021-05-31,50130620,1910,135,2021,112


In [19]:
merge_inner = pd.merge(data1_result, data2_result)
merge_inner

,base_date,emd_cd,fe_resid_reg_pop,fe_foreign_pop,m_resid_reg_pop,m_foreign_pop
0,2018-01-31,50110250,10081,1338,10808,2192
1,2018-02-28,50110250,10089,1337,10796,2205
2,2018-03-31,50110250,10136,1328,10860,2208
3,2018-04-30,50110250,10168,1357,10868,2213
4,2018-05-31,50110250,10214,1368,10898,2235
...,...,...,...,...,...,...
1801,2021-02-28,50130620,1904,137,2013,119
1802,2021-03-31,50130620,1918,137,2007,116
1803,2021-04-30,50130620,1904,135,2004,118
1804,2021-05-31,50130620,1910,135,2021,112


In [20]:
merge_inner.to_csv("resident_pop_processing2(sex).csv", index_label=False)

In [21]:
df = pd.read_csv("resident_pop_processing2(sex).csv")
df

,base_date,emd_cd,fe_resid_reg_pop,fe_foreign_pop,m_resid_reg_pop,m_foreign_pop
0,2018-01-31,50110250,10081,1338,10808,2192
1,2018-02-28,50110250,10089,1337,10796,2205
2,2018-03-31,50110250,10136,1328,10860,2208
3,2018-04-30,50110250,10168,1357,10868,2213
4,2018-05-31,50110250,10214,1368,10898,2235
...,...,...,...,...,...,...
1801,2021-02-28,50130620,1904,137,2013,119
1802,2021-03-31,50130620,1918,137,2007,116
1803,2021-04-30,50130620,1904,135,2004,118
1804,2021-05-31,50130620,1910,135,2021,112
